In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"/Users/shengji/Documents/GitHub/Room-Navigation/Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using QMDP


(process:77498): Gtk-WARNING **: 20:19:45.799: Locale not supported by C library.
	Using the fallback 'C' locale.
┌ Info: Recompiling stale cache file /Users/shengji/.julia/compiled/v1.0/QMDP/vYw0A.ji for QMDP [3aa3ecc9-5a5d-57c8-8188-3e47bd8068d2]
└ @ Base loading.jl:1190


In [3]:
num_x_pts = 50 # e.g. 50
num_y_pts = 50 # e.g. 50
num_th_pts = 20 # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);

In [4]:
vlist = [0, 1, 2, 3]
omlist = [-1, -0.5, 0, 0.5, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [5]:
cut_points = [1, 2, 3, 5, 10, 20, 50]
sensor = DiscreteLidar(cut_points);

In [6]:
config = 3 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

In [7]:
num_particles = 2000
resampler = LidarResampler(num_particles, LowVarianceResampler(num_particles))
# for the bumper environment
# resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [9]:
solver = QMDPSolver(max_iterations=20, tolerance=1e-3, verbose=true)
p = solve(solver, m);

[Iteration 1   ] residual:        9.9 | iteration runtime:  63686.201 ms, (      63.7 s total)
[Iteration 2   ] residual:        9.4 | iteration runtime: 121572.119 ms, (       185 s total)
[Iteration 3   ] residual:        4.3 | iteration runtime: 127732.432 ms, (       313 s total)


InterruptException: InterruptException:

In [39]:
p.action_map

20-element Array{RoombaAct,1}:
 [0.0, -1.0]
 [1.0, -1.0]
 [2.0, -1.0]
 [3.0, -1.0]
 [0.0, -0.5]
 [1.0, -0.5]
 [2.0, -0.5]
 [3.0, -0.5]
 [0.0, 0.0] 
 [1.0, 0.0] 
 [2.0, 0.0] 
 [3.0, 0.0] 
 [0.0, 0.5] 
 [1.0, 0.5] 
 [2.0, 0.5] 
 [3.0, 0.5] 
 [0.0, 1.0] 
 [1.0, 1.0] 
 [2.0, 1.0] 
 [3.0, 1.0] 

In [40]:
println(sizeof(p.alphas[1]))

1200000


In [31]:
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

MethodError: MethodError: POMDPs.action(::AlphaVectorPolicy{RoombaPOMDP{DiscreteLidar,Int64},RoombaAct}, ::ParticleCollection{RoombaState}) is ambiguous. Candidates:
  action(p::Policy, b::ParticleCollection{RoombaState}) in Main at In[29]:2
  action(p::AlphaVectorPolicy, b::AbstractParticleBelief) in ParticleFilters at /Users/shengji/.julia/packages/ParticleFilters/oMKH3/src/policies.jl:21
Possible fix, define
  action(::AlphaVectorPolicy, ::ParticleCollection{RoombaState})

In [17]:
using JLD
JLD.save("QMDP_Lidar_policy.jld", "policy", p)

┌ Info: Recompiling stale cache file /Users/shengji/.julia/compiled/v1.0/JLD/l8ijy.ji for JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base loading.jl:1190


In [3]:
d = JLD.load("QMDP_Lidar_policy.jld")

┌ Warning: type POMDPPolicies.AlphaVectorPolicy{AA228FinalProject.RoombaPOMDP{AA228FinalProject.DiscreteLidar,Core.Int64},AA228FinalProject.RoombaAct} not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type AA228FinalProject.RoombaPOMDP{AA228FinalProject.DiscreteLidar,Core.Int64} not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type AA228FinalProject.DiscreteLidar not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type AA228FinalProject.RoombaMDP{AA228FinalProject.DiscreteRoombaStateSpace,Core.Array{AA228FinalProject.RoombaAct,1}} not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type AA228FinalProject.Room not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_

Dict{String,Any} with 1 entry:
  "policy" => ##POMDPPolicies.AlphaVectorPolicy{AA228FinalProject.RoombaPOMDP{A…